In [4]:
import time
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

In [14]:
###### INIT ######
init_country = 'ru' # ua,au,at,az,al,ar,aw,be,by,bg,ru
init_category = 16 # films
init_genre = 'melodrama' # comedy,action,detective,melodrama,thriller,horror,adventures,sport,fiction,crime
init_year = '2021,2020,2016-2019' # 2021,2020,2016-2019,2010-2015,2000-2009,1990-1999,1980-1989,1970-1979,-1969

url_sampler = lambda country = init_country, genre = init_genre, year = init_year: \
    f"""https://megogo.net/ua/search-extended?category_id=16&country_id={country}&main_tab=filters&sort=popular&video_genre_id={genre}&year_id={year}"""

target_url = url_sampler()

print('default - ', url_sampler())
print('custom - ', url_sampler('en', 'adventure', '2017'))

default -  https://megogo.net/ua/search-extended?category_id=16&country_id=ru&main_tab=filters&sort=popular&video_genre_id=melodrama&year_id=2021
custom -  https://megogo.net/ua/search-extended?category_id=16&country_id=en&main_tab=filters&sort=popular&video_genre_id=adventure&year_id=2017


In [15]:
def try_search(driver, target):
    """Trying to find LOAD MORE button. If not exis, return False. List of items is full

    Args:
        driver (_type_): _description_
        target (_type_): _description_

    Returns:
        _type_: _description_
    """
    try:
        driver.find_element_by_class_name(target).click()
        return True
    except:
        return False

In [16]:
##### For remote selenium server only #####

##### To make this work, run :

# wget https://github.com/SeleniumHQ/selenium/releases/download/selenium-4.1.0/selenium-server-4.1.2.jar

# wget https://chromedriver.storage.googleapis.com/98.0.4758.80/chromedriver_linux64.zip

# unzip chromedriver_linux64.zip

# sudo mv -f chromedriver /usr/bin/chromedriver

# wget https://dl.google.com/linux/direct/google-chrome-stable_current_x86_64.rpm

# yum localinstall google-chrome-stable_current_x86_64.rpm

# java -jar selenium-server-4.1.2.jar standalone --allow-cors true - Start selenium remote server 

options = webdriver.ChromeOptions()
options.add_argument("no-sandbox")
options.add_argument("--headless")
options.add_argument("--disable-gpu")

driver = webdriver.Remote(
    command_executor='http://172.30.110.112:4444',
    options=options
)
driver.get(target_url)
target_div = 'pagination-more'

while try_search(driver, target_div):
    time.sleep(1)

items = driver.find_elements_by_class_name('videoItem')
html_items = [item.get_attribute("outerHTML") for item in items]

driver.quit()

/tmp/ipykernel_31894/3284241410.py:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_class_name(target).click()
/tmp/ipykernel_31894/4196354492.py:34: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  items = driver.find_elements_by_class_name('videoItem')


In [ ]:

# ! Названия, год, страна, айтем url , / жанр 

In [17]:
def parse_megogo(html_item, parse_country: True):
    """Parsing html data from Megogo site. If parse_country passed True 
    and items list are big, site may block requests.

    Args:
        html_item (_type_): _description_
        parse_country (True): _description_

    Returns:
        _type_: _description_
    """
    try:
        obj = BeautifulSoup(html_item, 'lxml')
    except Exception (e):
        print(e)
        return [None] * 5 
    
    if parse_country:
        try:
            obj_page = BeautifulSoup(requests.get(list(obj.a.attrs.values())[0]).text, 'lxml')
            country = obj_page.findAll('a', {'class' : 'video-country link-default'})[0].text
        except:
            country = None
    else:
        country = None
        
    try:
        name, year = list(obj.a.attrs.values())
    except:
        name, year = None, None
        
    try:
        item_url, genre = obj.findAll('span', {'class' :  'video-country'})[0].text, \
            obj.findAll('span', {'class' :  'video-year'})[0].text
    except:
        item_url, genre = None, None
    
    return [name, year, country, item_url, genre]

In [18]:
res = pd.DataFrame([parse_megogo(item, parse_country = True) for item in html_items], \
                columns = ['item_url', 'name', 'country', 'genre', 'year'])

In [19]:
res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   item_url  7 non-null      object
 1   name      7 non-null      object
 2   country   7 non-null      object
 3   genre     7 non-null      object
 4   year      7 non-null      object
dtypes: object(5)
memory usage: 408.0+ bytes


In [13]:
path = '/home/vmalyniak/home/jupyter/parse_megogo/'
res.to_csv(path + f'megogo_export_{time.ctime()}.csv')